In [1]:
import tensorflow as tf
from cpsrl.models.mean import ConstantMean, LinearMean
from cpsrl.models.covariance import EQ
from cpsrl.models.gp import VFEGP
from cpsrl.models.gp import VFEGPStack

In [2]:
# Check constant mean works
constant_mean = ConstantMean(trainable=True,
                             dtype=tf.float64)

ones = tf.ones(shape=(2, 3), dtype=tf.float64)
constant_mean(ones)

# Check linear mean works
linear_mean = LinearMean(input_dim=3,
                         trainable=True,
                         dtype=tf.float64)
linear_mean(ones)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 0.])>

In [3]:
# Check EQ covariance works
log_coeff = 0.
log_scales = [-1., -1.]

# Check EQ kernel works
eq = EQ(log_coeff=log_coeff,
        log_scales=log_scales,
        trainable=True,
        dtype=tf.float64)

x1 = tf.random.uniform(shape=(4, 2), dtype=tf.float64)
x2 = tf.random.uniform(shape=(3, 2), dtype=tf.float64)

eq(x1, x2)

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[0.12742794, 0.82043448, 0.30216512],
       [0.27389163, 0.28310373, 0.87914323],
       [0.03224904, 0.109014  , 0.00734735],
       [0.13637333, 0.6393487 , 0.51403885]])>

In [4]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

mean = LinearMean(input_dim=(state_dim+action_dim),
                  trainable=trainable_mean,
                  dtype=dtype)

cov = EQ(log_coeff=log_coeff,
         log_scales=log_scales,
         trainable=trainable_cov,
         dtype=dtype)

vfe_gp = VFEGP(mean=mean,
               cov=cov,
               x_train=x_train,
               y_train=y_train[:, :1],
               x_ind=None,
               ind_fraction=0.1,
               trainable_inducing=trainable_inducing,
               log_noise=log_noise,
               trainable_noise=trainable_noise,
               dtype=dtype) 

vfe_gp.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-910.2239764424369>

In [5]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

means = [LinearMean(input_dim=(state_dim+action_dim),
                    trainable=trainable_mean,
                    dtype=dtype) for i in range(state_dim)]

covs = [EQ(log_coeff=log_coeff,
           log_scales=log_scales,
           trainable=trainable_cov,
           dtype=dtype) for i in range(state_dim)]

vfe_gps = [VFEGP(mean=means[i],
                 cov=covs[i],
                 x_train=x_train,
                 y_train=y_train[:, i:i+1],
                 x_ind=None,
                 ind_fraction=0.1,
                 trainable_inducing=trainable_inducing,
                 log_noise=log_noise,
                 trainable_noise=trainable_noise,
                 dtype=dtype) 
           for i in range(state_dim)]

vfe_gp_stack = VFEGPStack(vfe_gps=vfe_gps,
                          dtype=dtype)

vfe_gp_stack.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-1680.944491665882>